In [6]:
# connect to postgres notification table.
DB_HOST = '10.14.39.97'
DB_NAME = 'postgres'
DB_USER = 'postgres'
DB_PASSWORD = 'Trellix123'

# main.py
import psycopg2
import json
from psycopg2 import extras
from pprint import pprint
import pandas as pd
import sys, os
# sys.path.append(os.path.abspath('../utils'))
# print(sys.path)
from mapping import mapping_malware_object, mapping_web_infection

# Database connection settings

# Establish a connection to the database
def connect_to_db():
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        print("Connected to the database successfully!")
        return conn
    except psycopg2.Error as e:
        print(f"Failed to connect to the database: {e}")
        return None

# Retrieve data from the notification table
def get_notification_data(conn):
    try:
        cur = conn.cursor(cursor_factory=extras.DictCursor)
        cur.execute("SELECT * FROM notifications limit 5")
        rows = cur.fetchall()
        print("Notification data retrieved successfully!")
        notifications = []
        for row in rows:
            # Convert the row to a dictionary
            notification = dict(row)
            # Load JSON data from the 'data' column into a dictionary
            # notification['data'] = json.loads(notification['data'])
            notifications.append(notification)
        return notifications
    except psycopg2.Error as e:
        print(f"Failed to retrieve notification data: {e}")
        return None
def normalize_data(data, path):
    if path in data:
        if isinstance(data[path], list):
            return pd.json_normalize(data, record_path=path)
        else:
            return pd.json_normalize([data[path]])
    else:
        return None
    
# Main function
def main():
    conn = connect_to_db()
    if conn:
        notifications = get_notification_data(conn)
        if notifications:
            for notification in notifications:
                try:
                    alert_data = json.loads(notification.get('data'))
                    alert_name = alert_data['alert']['name']
                    if alert_name == 'web-infection':
                        mapping = mapping_web_infection
                    if alert_name == 'malware-object':
                        mapping = mapping_malware_object
                    else:
                        print(f"Not sure what is the alert type..")
                    flat_data = normalize_data(alert_data, ['alert', 'explanation', 'malware-detected', 'malware'])
                    if flat_data is not None:
                        flat_data = flat_data.rename(columns=lambda x: f"malware_{x}")

                    cnc_services_data = normalize_data(alert_data, ['alert', 'explanation', 'cnc-services', 'cnc-service'])
                    if cnc_services_data is not None:
                        cnc_services_data = cnc_services_data.rename(columns=lambda x: f"cnc_services_{x}")

                    # Merge the dataframes
                    if flat_data is not None and cnc_services_data is not None:
                        flat_data = pd.merge(flat_data, cnc_services_data, left_index=True, right_index=True)
                    elif flat_data is not None:
                        flat_data = flat_data
                    elif cnc_services_data is not None:
                        flat_data = cnc_services_data
                    else:
                        flat_data = None

                    # Map the data to the desired format
                    if flat_data is not None:
                        mapped_dict = {}
                        for key, value in mapping.items():
                            if key == 'malware' or key == 'cnc_services':
                                continue
                            mapped_dict[key] = flat_data[value].iloc[0]

                        # Handle the malware and cnc-services lists
                        # Handle the malware and cnc-services lists
                        if 'malware' in mapping:
                            malware_dict = {}
                            for key, value in mapping['malware'].items():
                                malware_dict[key] = flat_data[value].iloc[0]
                            mapped_dict['malware'] = malware_dict

                        if 'cnc_services' in mapping:
                            cnc_services_dict = {}
                            for key, value in mapping['cnc_services'].items():
                                cnc_services_dict[key] = flat_data[value].iloc[0]
                            mapped_dict['cnc_services'] = cnc_services_dict

                        product = mapped_dict['product']
                        name = mapped_dict['alert_name']
                        print(f"{product}_{name}")
                    else:
                        print("No data found")
                except Exception as e:
                    print(notification.get('data'))
                    print(f"failed to process: {e}")
                    break
        conn.close()

if __name__ == "__main__":
    main()

Connected to the database successfully!
Notification data retrieved successfully!
{
  "appliance-id": "000BABC9615A",
  "product": "Web MPS",
  "version": "10.0.1.996666",
  "appliance": "tanuj-4500-2.eng.fireeye.com",
  "msg": "normal",
  "alert": {
    "id": "39",
    "name": "malware-object",
    "uuid": "b628ad2e-a2ba-467e-b4a9-4b9bd362e466",
    "ack": "no",
    "sc-version": "1492.130",
    "severity": "majr",
    "explanation": {
      "analysis": "binary",
      "protocol": "tcp",
      "malware-detected": {
        "malware": {
          "sid": "1234",
          "name": "Trellix-TestEvent-SIG",
          "stype": "generated-content",
          "type": "pdf",
          "profile": "winxp-sp2-rp",
          "md5sum": "d41d8cd98f00b204e9800998ecf8427e",
          "application": "explorer",
          "user": "testfire",
          "original": "TRELLIX-TEST-FIRE",
          "downloaded-at": "2024-07-22T21:54:32Z",
          "submitted-at": "2024-07-22T21:54:30Z",
          "executed-

In [1]:
import sys, os
sys.path.append(os.path.abspath('../utils'))
print(sys.path)

['/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python39.zip', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/lib-dynload', '', '/Users/tanuj.maheshwari/Documents/GitHub/alert-portal/.venv/lib/python3.9/site-packages', '/Users/tanuj.maheshwari/Documents/GitHub/alert-portal/utils']


In [23]:
import json

def flatten_json(data, prefix=''):
    result = {}
    for key, value in data.items():
        new_key = prefix + key if prefix else key
        if isinstance(value, dict):
            result.update(flatten_json(value, new_key + '.'))
        else:
            result[new_key] = value
    return result

# Load JSON data
json_data = '''{
    "appliance-id": "000BABC9615A",
    "product": "Web MPS",
    "version": "10.0.1.996666",
    "appliance": "tanuj-4500-2.eng.fireeye.com",
    "msg": "normal",
    "alert": {
        "id": "39",
        "name": "malware-object",
        "uuid": "b628ad2e-a2ba-467e-b4a9-4b9bd362e466",
        "ack": "no",
        "sc-version": "1492.130",
        "severity": "majr",
        "explanation": {
            "analysis": "binary",
            "protocol": "tcp",
            "malware-detected": {
                "malware": {
                    "sid": "1234",
                    "name": "Trellix-TestEvent-SIG",
                    "stype": "generated-content",
                    "type": "pdf",
                    "profile": "winxp-sp2-rp",
                    "md5sum": "d41d8cd98f00b204e9800998ecf8427e",
                    "application": "explorer",
                    "user": "testfire",
                    "original": "TRELLIX-TEST-FIRE",
                    "downloaded-at": "2024-07-22T21:54:32Z",
                    "submitted-at": "2024-07-22T21:54:30Z",
                    "executed-at": "2024-07-22T21:54:32Z"
                }
            },
            "cnc-services": {
                "cnc-service": {
                    "port": "200",
                    "protocol": "tcp",
                    "type": "VmsigMatch",
                    "sid": "1234",
                    "sname": "Trellix-TestEvent-SIG",
                    "address": "Trellix-TestEvent.example.com",
                    "channel": "Trellix-TestEvent Channel 1"
                }
            }
        },
        "src": {
            "vlan": "0",
            "ip": "169.250.0.1",
            "mac": "00:11:33:55:77:99",
            "port": "10"
        },
        "dst": {
            "ip": "127.0.0.20",
            "mac": "00:22:44:66:88:aa",
            "port": "20"
        },
        "action": "notified",
        "alert-url": "https://tanuj-4500-2.eng.fireeye.com/detection/objects?uuid=b628ad2e-a2ba-467e-b4a9-4b9bd362e466",
        "occurred": "2024-07-22T21:54:30Z"
    }
}'''

data = json.loads(json_data)

# Flatten JSON data
flat_data = flatten_json(data, '.')
pprint(flat_data)
# Accessing values
print(flat_data['appliance-id'])  # Output: 000BABC9615A
print(flat_data['alert.id'])  # Output: 39
print(flat_data['alert.explanation.malware-detected.malware.sid'])  # Output: 1234

{'.alert.ack': 'no',
 '.alert.action': 'notified',
 '.alert.alert-url': 'https://tanuj-4500-2.eng.fireeye.com/detection/objects?uuid=b628ad2e-a2ba-467e-b4a9-4b9bd362e466',
 '.alert.dst.ip': '127.0.0.20',
 '.alert.dst.mac': '00:22:44:66:88:aa',
 '.alert.dst.port': '20',
 '.alert.explanation.analysis': 'binary',
 '.alert.explanation.cnc-services.cnc-service.address': 'Trellix-TestEvent.example.com',
 '.alert.explanation.cnc-services.cnc-service.channel': 'Trellix-TestEvent '
                                                        'Channel 1',
 '.alert.explanation.cnc-services.cnc-service.port': '200',
 '.alert.explanation.cnc-services.cnc-service.protocol': 'tcp',
 '.alert.explanation.cnc-services.cnc-service.sid': '1234',
 '.alert.explanation.cnc-services.cnc-service.sname': 'Trellix-TestEvent-SIG',
 '.alert.explanation.cnc-services.cnc-service.type': 'VmsigMatch',
 '.alert.explanation.malware-detected.malware.application': 'explorer',
 '.alert.explanation.malware-detected.malware.downl

KeyError: 'appliance-id'

In [22]:
import pandas as pd
import json

# Load JSON data
json_data = '''{
    "appliance-id": "000BABC9615A",
    "product": "Web MPS",
    "version": "10.0.1.996666",
    "appliance": "tanuj-4500-2.eng.fireeye.com",
    "msg": "normal",
    "alert": {
        "id": "39",
        "name": "malware-object",
        "uuid": "b628ad2e-a2ba-467e-b4a9-4b9bd362e466",
        "ack": "no",
        "sc-version": "1492.130",
        "severity": "majr",
        "explanation": {
            "analysis": "binary",
            "protocol": "tcp",
            "malware-detected": {
                "malware": {
                    "sid": "1234",
                    "name": "Trellix-TestEvent-SIG",
                    "stype": "generated-content",
                    "type": "pdf",
                    "profile": "winxp-sp2-rp",
                    "md5sum": "d41d8cd98f00b204e9800998ecf8427e",
                    "application": "explorer",
                    "user": "testfire",
                    "original": "TRELLIX-TEST-FIRE",
                    "downloaded-at": "2024-07-22T21:54:32Z",
                    "submitted-at": "2024-07-22T21:54:30Z",
                    "executed-at": "2024-07-22T21:54:32Z"
                }
            },
            "cnc-services": {
                "cnc-service": {
                    "port": "200",
                    "protocol": "tcp",
                    "type": "VmsigMatch",
                    "sid": "1234",
                    "sname": "Trellix-TestEvent-SIG",
                    "address": "Trellix-TestEvent.example.com",
                    "channel": "Trellix-TestEvent Channel 1"
                }
            }
        },
        "src": {
            "vlan": "0",
            "ip": "169.250.0.1",
            "mac": "00:11:33:55:77:99",
            "port": "10"
        },
        "dst": {
            "ip": "127.0.0.20",
            "mac": "00:22:44:66:88:aa",
            "port": "20"
        },
        "action": "notified",
        "alert-url": "https://tanuj-4500-2.eng.fireeye.com/detection/objects?uuid=b628ad2e-a2ba-467e-b4a9-4b9bd362e466",
        "occurred": "2024-07-22T21:54:30Z"
    }
}'''

data = json.loads(json_data)

# Flatten JSON data
flat_data = pd.json_normalize(data)

# Convert to dictionary
flat_dict = flat_data.to_dict(orient='records')[0]

# Accessing values
print(flat_dict['appliance-id'])  # Output: 000BABC9615A
print(flat_dict['alert.id'])  # Output: 39
print(flat_dict['alert.explanation.malware-detected.malware.sid'])  # Output: 1234

000BABC9615A
39
1234


In [21]:
import json
from flatten_json import flatten

# Load JSON data
json_data = '''{
    "appliance-id": "000BABC9615A",
    "product": "Web MPS",
    "version": "10.0.1.996666",
    "appliance": "tanuj-4500-2.eng.fireeye.com",
    "msg": "normal",
    "alert": {
        "id": "39",
        "name": "malware-object",
        "uuid": "b628ad2e-a2ba-467e-b4a9-4b9bd362e466",
        "ack": "no",
        "sc-version": "1492.130",
        "severity": "majr",
        "explanation": {
            "analysis": "binary",
            "protocol": "tcp",
            "malware-detected": {
                "malware": {
                    "sid": "1234",
                    "name": "Trellix-TestEvent-SIG",
                    "stype": "generated-content",
                    "type": "pdf",
                    "profile": "winxp-sp2-rp",
                    "md5sum": "d41d8cd98f00b204e9800998ecf8427e",
                    "application": "explorer",
                    "user": "testfire",
                    "original": "TRELLIX-TEST-FIRE",
                    "downloaded-at": "2024-07-22T21:54:32Z",
                    "submitted-at": "2024-07-22T21:54:30Z",
                    "executed-at": "2024-07-22T21:54:32Z"
                }
            },
            "cnc-services": {
                "cnc-service": {
                    "port": "200",
                    "protocol": "tcp",
                    "type": "VmsigMatch",
                    "sid": "1234",
                    "sname": "Trellix-TestEvent-SIG",
                    "address": "Trellix-TestEvent.example.com",
                    "channel": "Trellix-TestEvent Channel 1"
                }
            }
        },
        "src": {
            "vlan": "0",
            "ip": "169.250.0.1",
            "mac": "00:11:33:55:77:99",
            "port": "10"
        },
        "dst": {
            "ip": "127.0.0.20",
            "mac": "00:22:44:66:88:aa",
            "port": "20"
        },
        "action": "notified",
        "alert-url": "https://tanuj-4500-2.eng.fireeye.com/detection/objects?uuid=b628ad2e-a2ba-467e-b4a9-4b9bd362e466",
        "occurred": "2024-07-22T21:54:30Z"
    }
}'''

data = json.loads(json_data)

# Flatten JSON data
flat_data = flatten(data)

# Accessing values
print(flat_data['appliance-id'])  # Output: 000BABC9615A
print(flat_data['alert_id'])  # Output: 39
pprint(flat_data)
print(flat_data['alert.explanation.malware-detected.malware.sid'])  # Output: 1234

000BABC9615A
39
{'alert_ack': 'no',
 'alert_action': 'notified',
 'alert_alert-url': 'https://tanuj-4500-2.eng.fireeye.com/detection/objects?uuid=b628ad2e-a2ba-467e-b4a9-4b9bd362e466',
 'alert_dst_ip': '127.0.0.20',
 'alert_dst_mac': '00:22:44:66:88:aa',
 'alert_dst_port': '20',
 'alert_explanation_analysis': 'binary',
 'alert_explanation_cnc-services_cnc-service_address': 'Trellix-TestEvent.example.com',
 'alert_explanation_cnc-services_cnc-service_channel': 'Trellix-TestEvent '
                                                       'Channel 1',
 'alert_explanation_cnc-services_cnc-service_port': '200',
 'alert_explanation_cnc-services_cnc-service_protocol': 'tcp',
 'alert_explanation_cnc-services_cnc-service_sid': '1234',
 'alert_explanation_cnc-services_cnc-service_sname': 'Trellix-TestEvent-SIG',
 'alert_explanation_cnc-services_cnc-service_type': 'VmsigMatch',
 'alert_explanation_malware-detected_malware_application': 'explorer',
 'alert_explanation_malware-detected_malware_downlo

KeyError: 'alert.explanation.malware-detected.malware.sid'